In [21]:
workspace()

El proyecto final para la clase de métodos númericos avanzados, descrito en este notebook, tratará con la problemática de las soluciones a ecuaciones lineales con intervalos desde varios métodos de soluciones directos e iterativos, con la finalidad de comparar la fiabilidad, la convergencia y la sencillez de cada uno.

Los métodos a implementar son, el directo (eliminación de Gauss), la versión linear de Krawczyk y el Hansel-Sengupta. Por último se intentará extender lo antes descrito para una ecuación de eigenvalores.

Una aplicación directa de este tipode problemas es el caso en el las resistencias de una mallado eléctrico son tomadas con su toleracia incluida, y no sólo con un número fijo, es decir, con un intervalo. Y, Por lo tanto, las corrientes resultado también estarán dadas por un intervalo.

In [108]:
using Intervalos

In [262]:
function -(x::Interval) #Había olvidado definir estas operación en el módulo
    return Interval(-x.right, -x.left)
end

function midpoint(N::Interval) #PuntoMedio
      return (N.left+N.right)/2
end

midpoint (generic function with 1 method)

In [180]:
type IntervalVector #Defino un Vector de intervalos
    row::Vector{Interval}
end


function Base.length(x::IntervalVector) #Cuantos intervalos contiene el vector 
    length(x.row)
end

function midpoint(x::IntervalVector) #Punto medio de un vector de intervalos
    mid=Real[]
    for i=1:length(x)
        push!(mid,midpoint(x.row[i]))
    end
    return mid
end

midpoint (generic function with 3 methods)

In [182]:
type IntervalMatrix #Defino una matrix de intervalos
    rows::Vector{IntervalVector}
end

#Definí una matriz de intervalos donde las columnas están descritas por cada vector proporcionado

function midpoint(M::IntervalMatrix)
    mid1=Vector[]
    for i=1:length(M.rows)
        push!(mid1,midpoint(M.rows[i]))
    end
    return mid1
end

midpoint (generic function with 3 methods)

In [192]:
A.rows[1]

IntervalVector([Interval(1800,2200),Interval(-1100,-900)])

In [ ]:
function *(M::IntervalMatrix, N::IntervalMatrix)
    if length(M.rows)==length(N.rows[1])
        P=Array[]
        for i=1:length(M.rows)
            #Encuentro cierta dificultad para definir las operaciones de matrices de intervalos, por lo cual, 
            #intentaré otra manera de definir mis matrices           

Intentemos resolver por método directo (eliminación de Gauss)

In [88]:
#GaussianElimination 2x2

function GaussianElimination(A::IntervalMatrix, b::IntervalVector)
    
    
    a11=A.rows[1].row[1]
    a21=A.rows[1].row[2]
    a12=A.rows[2].row[1]
    a22=A.rows[2].row[2]
    
    a=a22-(a21/a11)*a12
    d=b.row[2]-(a21/a11)*b.row[1]
    x2=d/a

    c=b.row[1]-a12*x2
    x1=c/A.rows[1].row[1]
    
    return IntervalVector([x1,x2])
end

GaussianElimination (generic function with 1 method)

In [259]:
R1=Interval(900,1100)
R2=Interval(900,1100)
R3=Interval(900,1100)

V1=Interval(10)
V2=Interval(5);

In [345]:
a1=IntervalVector([R1 + R2, -R2])
a2=IntervalVector([-R2, R2 + R3])
A=IntervalMatrix([a1, a2])

b=IntervalVector([V1, -V2]);

LoadError: type: apply: expected Function, got Type{Array{Interval,1}}
while loading In[345], in expression starting on line 1

In [91]:
GaussianElimination(A,b)

IntervalVector([Interval(0.004227994227994228,0.006029786400156771),Interval(-0.0006349206349206357,0.0007760141093474431)])

Lo cual contiene a la solución exacta, es decir, la solución  que daría si no tuvieramos incertidumbre en las resitencias.

###Buscando la definición más eficiente y fácil de usar

In [251]:
workspace()
using Intervalos

In [308]:
typealias IntervalMatrix{Interval} Array{Interval,2}
#Esto hará que cuando cree funciones pueda llamar IntervalMatrix a lo que yo quiera y 
#julia detectará que es un Array{Interval,2}  

typealias IntervalVector{Interval} Array{Interval,1};

In [264]:
A=[R1+R2 -R2;-R2 R2+R3]

2x2 Array{Interval,2}:
 Interval(1800,2200)   Interval(-1100,-900)
 Interval(-1100,-900)  Interval(1800,2200) 

In [346]:
b=[V1;-V2]

2-element Array{Interval,1}:
 Interval(10,10)
 Interval(-5,-5)

In [313]:
B=[Interval(1) Interval(2); Interval(1) Interval(2)]

2x2 Array{Interval,2}:
 Interval(1,1)  Interval(2,2)
 Interval(1,1)  Interval(2,2)

In [314]:
A*B #No tengo que definir las operaciones entre matrices de intervalos, julia ya sabe como multiplicar matrices
# y mi modulo ya sabe como hacer las operaciones con intervalos

2x2 Array{Interval,2}:
 Interval(700,1300)  Interval(1400,2600)
 Interval(700,1300)  Interval(1400,2600)

In [305]:
function midpoint(M::IntervalMatrix)
    x=zeros(Real,size(M)[1],size(M)[2])
    for i=1:size(M)[1]
        for j=1:size(M)[2]
            x[i,j]=midpoint(M[i,j])
        end
    end
    return x
end

midpoint (generic function with 2 methods)

In [306]:
midpoint(A)

2x2 Array{Real,2}:
  2000.0  -1000.0
 -1000.0   2000.0

In [348]:
#GaussianElimination 2x2

function GaussianElimination(A::IntervalMatrix, b::IntervalVector)
    

    a=A[2,2]-(A[2,1]/A[1,1])*A[1,2]
    d=b[2]-(A[2,1]/A[1,1])*b[1]
    x2=d/a

    c=b[1]-A[1,2]*x2
    x1=c/A[1,1]
    
    
    X=[x1,x2]
    return X
end

#Evidentemente esta forma de nombrar es mucho más fácil de usar, en cuanto a indices y en cuanto a operaciones, y
# por lo tanto, más fácil de manipular

GaussianElimination (generic function with 1 method)

In [349]:
GaussianElimination(A,b)

2-element Array{Interval,1}:
 Interval(0.004227994227994228,0.006029786400156771)   
 Interval(-0.0006349206349206357,0.0007760141093474431)

##Métodos aproximados

Linear Krawczyk Method...